In [143]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import requests

In [144]:
dic_cols = {'country':'pais','type':'tipo','year':'anio','consumption':'cons','production':'produccion','gdp':'pbi','population':'poblacion','intensity':'intensidad','emission':'emision_co2','indicator':'indicador','fuel':'combustible','wind':'eolica','electricity':'elec','energy':'energia','iso_code':'pais_iso','share':'participacion','renewable':'renovable','oil':'petroleo','change':'cambio','coal':'carbon','time':'periodo','product':'producto','value':'valor','unit':'unidad','iso3':'pais_iso','source':'fuente'}

In [145]:
def nomCols(df,dic):
    for column in df.columns:
        df.rename(columns={column:column.lower()},inplace=True)
    for column in df.columns:
        for key in dic.keys():
            if column == key:
                nom = column.replace(column,dic[key])
                df.rename(columns={column:nom},inplace=True)
            elif key in column:
                nom = column.replace(key,dic[key])
                df.rename(columns={column:nom},inplace=True)
    return df

def dropBefore1980(df):
    lista = ['year','anio','año']
    for column in df.columns:
        if column.lower() in lista:
            df = df[df[column]>=1980].reset_index(drop=True)
    return df

**EnergyCO2**

In [8]:
dataset = pd.read_csv(r"..\energyco2.csv")
df = pd.DataFrame(dataset)

Dropeo la columna Unnamed: 0 que no contiene informacion

In [88]:
df = df.drop(columns="Unnamed: 0")

In [89]:
#Le paso una serie y un tipo para chequear si los elementos de la serie pertenecen a esa clase o no
def chequeoTipo(serie,clase):
    pertenece = False
    lista = []
    for item in serie:
        if isinstance(item,clase):
            pertenece = True
        else:
            lista.append(item)
    if len(lista)>0:
        return lista
    else: 
        return pertenece   


In [91]:
for item in df.columns:
    for i in range(0,df.shape[0]):
        if df[i,item]:
            print(f"{item}:{type(df[item][i])}")

KeyError: (0, 'Country')

In [92]:
chequeoTipo(df['CO2_emission'],float)

True

Todos los features tienen elementos de las clases adecuadas para sus datos

Renombro columnas usando snake_case en español según definido por el equipo de trabajo

Una vez que ya chequee que cada columna tenga el tipo de dato adecuado, puedo usar esa condicion para chequear todas las columnas y convertir a float de dos digitos

In [ ]:
for columna in df.columns:
    if isinstance(df[columna][1],float):
        df[columna] = df[columna].round(2)

<hr>

<h1>Climate Related Disasters Frequency</h1>
<h2><i>Version CSV</i></h2>

In [165]:
dataset = pd.read_csv(r"..\desastres_naturales\Climate-related_Disasters_Frequency.csv")
df2 = pd.DataFrame(dataset)

In [166]:
df2.head(2)

,ObjectId,Country,ISO2,ISO3,Indicator,Code,Unit,Source,F1980,F1981,...,F2012,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021
0,1,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",ECFRF_NUM,Number of,"The Emergency Events Database (EM-DAT) , Centr...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
1,2,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",ECFRF_NUM,Number of,"The Emergency Events Database (EM-DAT) , Centr...",NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
df2 = df2.drop(columns=['ObjectId','ISO2','Code','Unit'])

Convierto NaN en ceros (en este caso se trata de frecuencias)

In [168]:
df2.iloc[:,5:] = df2.iloc[:,5:].fillna(0)

Aplico la funcion para renombrar columnas nomCols usando el diccionario creado, dic_cols

In [169]:
nomCols(df2,dic_cols)

,pais,pais_iso,indicador,fuente,f1980,f1981,f1982,f1983,f1984,f1985,...,f2012,f2013,f2014,f2015,f2016,f2017,f2018,f2019,f2020,f2021
0,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",NaN,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",1.0,0.0,0.0,0.0,0.0,0.0,...,6.0,4.0,2.0,1.0,4.0,1.0,3.0,6.0,5.0,2.0
3,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",NaN,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,0.0,4.0,0.0,2.0,2.0,1.0,1.0,1.0
4,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,Zambia,ZMB,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",NaN,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,3.0,1.0
964,Zimbabwe,ZWE,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",NaN,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
965,Zimbabwe,ZWE,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
966,Zimbabwe,ZWE,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0


In [170]:
df2.iloc[:,5:] = df2.iloc[:,5:].fillna(0)
for column in df2.columns:
    if re.search('f\d{2,}', column):
        nom = column.split('f')[1]
        df2[column]=df2[column].round(2)
        df2.rename(columns={column:nom},inplace=True)

In [145]:
df2.indicador.unique()

array(['Climate related disasters frequency, Number of Disasters: Drought',
       'Climate related disasters frequency, Number of Disasters: Extreme temperature',
       'Climate related disasters frequency, Number of Disasters: Flood',
       'Climate related disasters frequency, Number of Disasters: Landslide',
       'Climate related disasters frequency, Number of Disasters: Storm',
       'Climate related disasters frequency, Number of Disasters: TOTAL',
       'Climate related disasters frequency, Number of Disasters: Wildfire'],
      dtype=object)

Recorto los datos de la columna indicador para mostrar unicamente los descriptores

In [146]:
df2['indicador'] = df2['indicador'].apply(lambda x: x.split(': ')[1])

In [148]:
df2.to_parquet(r"..\desastres_naturales\Climate-related_Disasters_Frequency_normalizado.parquet")

<hr>

<h1>OWID-Energy-Consumption-Source</h1>

In [73]:
dataset = pd.read_csv(r"..\energy_consumption\owid-energy-consumption-source.csv")
df3 = pd.DataFrame(dataset)

In [149]:
df3.head()

,pais,anio,pais_iso,poblacion,pbi,biocombustible_cons,biocombustible_elec,biocombustible_share_energia,carbon_intensidad_elec,carbon_cons,...,other_renovable_elec,other_renewable_exc_biocombustible_elec,primary_energia_cons,renovables_cons,renovables_elec,solar_cons,solar_elec,eolica_cons,eolica_elec,eolica_share_energia
0,Afghanistan,1980,AFG,13356500.0,1.532984e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1981,AFG,13171679.0,1.564534e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1982,AFG,12882518.0,1.598041e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1983,AFG,12537732.0,1.675533e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1984,AFG,12204306.0,1.707215e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [365]:
for i in range(3,128):
    if chequeoTipo(df3.iloc[:,i],float):
        continue
    else:
        print(f"{df3.iloc[0:1,i]}")

Renombro columnas y me quedo con datos de 1980 en adelante

In [77]:
nomCols(df3,dic_cols)
df3 = dropBefore1980(df3)

In [78]:
for columna in df3.columns:
    if isinstance(df3[columna][2],float):
        df3[columna] = df3[columna].round(2)

In [79]:
df3.shape

(12309, 128)

In [406]:
df3.dropna(thresh=4)

,pais,anio,pais_iso,poblacion,pbi,biocombustible_cons,biocombustible_elec,carbon_intensidad_elec,carbon_cons,carbon_elec,...,other_renovable_cons,other_renovable_elec,other_renovable_exc_biocombustible_elec,primary_energia_cons,renovables_cons,renovables_elec,solar_cons,solar_elec,eolica_cons,eolica_elec
0,Afghanistan,1980,AFG,13356500.0,1.532984e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,7.791,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1981,AFG,13171679.0,1.564534e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.777,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1982,AFG,12882518.0,1.598041e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.348,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1983,AFG,12537732.0,1.675533e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,11.436,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1984,AFG,12204306.0,1.707215e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,11.489,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12304,Zimbabwe,2017,ZWE,14236599.0,2.194784e+10,NaN,0.32,363.885,NaN,3.02,...,NaN,0.32,0.0,45.936,NaN,4.26,NaN,0.01,NaN,0.0
12305,Zimbabwe,2018,ZWE,14438812.0,2.271535e+10,NaN,0.39,358.160,NaN,3.69,...,NaN,0.39,0.0,47.502,NaN,5.40,NaN,0.01,NaN,0.0
12306,Zimbabwe,2019,ZWE,14645473.0,NaN,NaN,0.38,288.240,NaN,3.62,...,NaN,0.38,0.0,49.427,NaN,7.65,NaN,0.01,NaN,0.0
12307,Zimbabwe,2020,ZWE,14862927.0,NaN,NaN,0.36,279.028,NaN,3.44,...,NaN,0.36,0.0,NaN,NaN,7.63,NaN,0.01,NaN,0.0


Elijo columnas que brindan informacion de cambio y 'per_capita' (no necesito esa info en el analisis planteado e igualmente puedo calcularlo con los demás datos de ser necesario)

In [80]:
lista = []
for column in df3.columns:
    if '_pct' in column or 'cambio' in column or 'per_capita' in column or '_participacion_' in column:
        lista.append(column)

In [81]:
df3 = df3.drop(columns=lista)

In [150]:
df3.head()

,pais,anio,pais_iso,poblacion,pbi,biocombustible_cons,biocombustible_elec,biocombustible_share_energia,carbon_intensidad_elec,carbon_cons,...,other_renovable_elec,other_renewable_exc_biocombustible_elec,primary_energia_cons,renovables_cons,renovables_elec,solar_cons,solar_elec,eolica_cons,eolica_elec,eolica_share_energia
0,Afghanistan,1980,AFG,13356500.0,1.532984e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1981,AFG,13171679.0,1.564534e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1982,AFG,12882518.0,1.598041e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1983,AFG,12537732.0,1.675533e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1984,AFG,12204306.0,1.707215e+10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
df3.to_parquet(r"..\energy_consumption\owid-energy-consumption-source_normalizado.parquet")

<hr>

<h1>Energia Estadistica Mensual</h1>

In [50]:
dataset = pd.read_csv(r"..\energia_estadistica_mensual\MES_0522.csv")
df4 = pd.DataFrame(dataset)

In [51]:
df4.head(1)

,Country,Time,Balance,Product,Value,Unit
0,Australia,May 2022,Net Electricity Production,Electricity,21305.5276,GWh


In [52]:
nomCols(df4,dic_cols)

,pais,periodo,balance,producto,valor,unidad
0,Australia,May 2022,Net Electricity Production,Electricity,21305.5276,GWh
1,Australia,May 2022,Net Electricity Production,Total Combustible Fuels,15280.3708,GWh
2,Australia,May 2022,Net Electricity Production,"Coal, Peat and Manufactured Gases",10488.3089,GWh
3,Australia,May 2022,Net Electricity Production,Oil and Petroleum Products,168.8215,GWh
4,Australia,May 2022,Net Electricity Production,Natural Gas,4388.9337,GWh
...,...,...,...,...,...,...
122896,IEA Total,January 2010,Total Imports,Electricity,34846.5580,GWh
122897,IEA Total,January 2010,Total Exports,Electricity,32372.5830,GWh
122898,IEA Total,January 2010,Used for pumped storage,Electricity,6869.2530,GWh
122899,IEA Total,January 2010,Distribution Losses,Electricity,62000.7330,GWh


In [53]:
df4.shape

(122901, 6)

In [54]:
df4.dropna(subset='valor',inplace=True)

In [55]:
df4.isna().sum()

pais        0
periodo     0
balance     0
producto    0
valor       0
unidad      0
dtype: int64

In [10]:
pertenece = False
lista = []
for item in df4.iloc[:,4]:
    if isinstance(item,float):
        pertenece = True
    else:
        lista.append((item,df4.columns[i]))
if len(lista)>0:
    print(lista)
else: 
    print(pertenece) 

True


In [57]:
df4['periodo'] = pd.to_datetime(df4['periodo'])
df4['valor'] = df4['valor'].round(2)
df4['anio'] = df4['periodo'].dt.year
df4['mes'] = df4['periodo'].dt.month

In [58]:
df4 = df4[['anio','mes','pais','balance','producto','valor']].sort_values(by=['anio','mes'])
df4.rename(columns={'valor':'cons_energia_gwh'},inplace=True)


In [62]:
df4.to_parquet(r"C:\Users\ferch\Desktop\Henry\Proyecto Grupal\CO2-Emissions\datasets\energia_estadistica_mensual\MES_O522_normalizado.parquet")

<h1>Climate Related Disasters Frequency</h1>
<h2>Versión API</h2>

In [154]:
url = "https://services9.arcgis.com/weJ1QsnbMYJlCHdG/arcgis/rest/services/Indicator_11_1_Physical_Risks_Climate_related_disasters_frequency/FeatureServer/0/query?where=1%3D1&outFields=Country,ISO3,Indicator,Source,F1980,F1981,F1982,F1983,F1984,F1985,F1986,F1987,F1988,F1989,F1990,F1991,F1992,F1993,F1994,F1995,F1996,F1997,F1998,F1999,F2000,F2001,F2002,F2003,F2004,F2005,F2006,F2007,F2008,F2009,F2010,F2011,F2012,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021&outSR=4326&f=json"
response = requests.get(url).json()

In [155]:
df5 = pd.DataFrame()

In [69]:
lista = [item for item in response['features'][0]['attributes'].keys()]

In [156]:
for i in range(0,len(response['features'])):
    df = pd.DataFrame.from_dict([response['features'][i]['attributes']])
    data=(df5,df)
    df5=pd.concat(data)

In [157]:
df5.reset_index(drop=True,inplace=True)

In [158]:
nomCols(df5,dic_cols)

,pais,pais_iso,indicador,fuente,f1980,f1981,f1982,f1983,f1984,f1985,...,f2012,f2013,f2014,f2015,f2016,f2017,f2018,f2019,f2020,f2021
0,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",None,None,None,None,None,None,...,None,None,None,None,None,None,1,None,None,1
1,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",None,None,None,None,None,None,...,1,None,None,None,None,None,None,None,None,None
2,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",1,None,None,None,None,None,...,6,4,2,1,4,1,3,6,5,2
3,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",None,None,None,None,None,None,...,3,1,None,4,None,2,2,1,1,1
4,"Afghanistan, Islamic Rep. of",AFG,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",None,None,None,None,None,None,...,None,None,1,None,None,2,None,None,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,Zambia,ZMB,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",None,1,None,1,None,None,...,None,1,1,None,None,None,None,2,3,1
964,Zimbabwe,ZWE,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",None,1,None,None,None,None,...,None,1,None,None,None,1,None,None,None,1
965,Zimbabwe,ZWE,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",None,None,None,None,None,None,...,None,1,2,None,None,None,None,1,None,None
966,Zimbabwe,ZWE,"Climate related disasters frequency, Number of...","The Emergency Events Database (EM-DAT) , Centr...",None,None,None,None,None,None,...,None,None,None,1,1,1,None,1,None,1


In [163]:
df5.iloc[:,5:]

,f1981,f1982,f1983,f1984,f1985,f1986,f1987,f1988,f1989,f1990,...,f2012,f2013,f2014,f2015,f2016,f2017,f2018,f2019,f2020,f2021
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,6,4,2,1,4,1,3,6,5,2
3,0,0,0,0,0,0,1,0,0,0,...,3,1,0,4,0,2,2,1,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,2,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,1,0,1,0,0,0,0,0,1,1,...,0,1,1,0,0,0,0,2,3,1
964,1,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,1,0,0,0,1
965,0,0,0,0,0,0,0,0,0,0,...,0,1,2,0,0,0,0,1,0,0
966,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,0,1,0,1


Cambio nombres de columnas y aplico un fillna sobre los campos de frecuencias para luego aplicar un round

In [175]:
for column in df5.columns:
    if re.search('f\d{2,}', column):
        df5[column] = df5[column].fillna(0)
        nom = column.split('f')[1]
        df5[column] = df5[column].round(2)
        df5.rename(columns={column:nom},inplace=True)

In [178]:
df5['indicador'] = df2['indicador'].apply(lambda x: x.split(': ')[1])

In [182]:
df6 = df5.groupby(by='pais').sum().transpose()

In [210]:
x = df6.index
y=df6[['Argentina','United States']]

In [211]:
y

pais,Argentina,United States
1980,4,14
1981,0,8
1982,2,26
1983,4,18
1984,2,28
1985,4,30
1986,4,12
1987,6,18
1988,2,44
1989,0,34


In [215]:
df5.head()

,pais,pais_iso,indicador,fuente,1980,1981,1982,1983,1984,1985,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,"Afghanistan, Islamic Rep. of",AFG,Drought,"The Emergency Events Database (EM-DAT) , Centr...",0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,"Afghanistan, Islamic Rep. of",AFG,Extreme temperature,"The Emergency Events Database (EM-DAT) , Centr...",0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,"Afghanistan, Islamic Rep. of",AFG,Flood,"The Emergency Events Database (EM-DAT) , Centr...",1,0,0,0,0,0,...,6,4,2,1,4,1,3,6,5,2
3,"Afghanistan, Islamic Rep. of",AFG,Landslide,"The Emergency Events Database (EM-DAT) , Centr...",0,0,0,0,0,0,...,3,1,0,4,0,2,2,1,1,1
4,"Afghanistan, Islamic Rep. of",AFG,Storm,"The Emergency Events Database (EM-DAT) , Centr...",0,0,0,0,0,0,...,0,0,1,0,0,2,0,0,1,0
